<img style="width:20%;float: left; margin-right: 10px;" src="https://upload.wikimedia.org/wikipedia/en/a/ae/CERN_logo.svg"/>



#    2D images classification using Machine Leaning for brain tumors

## Introduction
Brain tumors are considered by the medical community, an aggressive and difficult to treat disease that affects adults and children.

The most common primary brain tumors are Gliomas, followed by Meningiomas and Pituitary.

Gliomas are the most common type of adult brain tumor, It is about 78 percent of malignant brain tumors according to the American Association of Neurological Surgeons.<a href="#1">[1]</a>

In this tutorial we are going to classify 2D images with machine learning, for brain tumor such as **meningioma**, **glioma** and **pituitary**, respect people with no tumors.

As mentioned in the preprocessing notebook, we are using like ilustrative example a public dataset, taken from 
kaggle <a href="#2">[2]</a>, this is a 2D MRI dataset.


## About ML for Brain Tumor Classification

Brain tumor classification is complex task, it requies sophistcate machine learning and math models using the newest in the state of the art for image processing like Convolutional Neural Networks <a href="#3">[3]</a>.
It also requires elaborated techniques to preprocess the images, such as skull stripping <a href="#4">[4]</a>, advanced tools for data normalization such as Ants <a href="#5">[5]</a>, tools for image transformation such as
scikit-image <a href="#6">[6]</a> or open computer vision library (open-cv) <a href="#7">[7]</a>

This tutorial is done using Tensorflow with Keras <a href="#8">[8]</a> and due the limited resources will a binary classifier using 64x64 pixels 2D images.



<hr>

# Let's get started!

The first step is to import the required modules. those modules are to handle numeric arrays, plotting, create the machine learning model and compute some statistics over the results.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import itertools
import glob

from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam #http://arxiv.org/abs/1412.6980
from tensorflow.keras.layers import Dense, Conv2D, Flatten, AveragePooling2D, Dropout,BatchNormalization,Activation,SpatialDropout2D
import tensorflow as tf
from tensorflow.keras import backend as K

from sklearn.metrics import auc
from sklearn.metrics import roc_curve
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.model_selection import train_test_split

# Defining and Select the categories

The second step is to defined the categories for our problem, in our case is an array of four labels for meningioma, glioma, pituitary tumors and not tumor.

We have to select two of those categories to perform the binary classification, initially we will take no tumor and glioma.

In [ ]:
categories = ['meningioma_tumor', 'glioma_tumor', 'pituitary_tumor', 'no_tumor']
categories_selected = ["no_tumor","glioma_tumor"]

# Utility function to plot a confisuion matrix

Confusion matrix is a table layout that allows to visualize the performance of the algorithm.<a href="#9">[9]</a>

This allows see :
* TP: True postive 
* TN: True negative
* FP: False positive
* FN: False negative


We took this code  snippet from kaggle <a href="#10">[10]</a> to produce a nice plot.


In [ ]:
def plot_confusion_matrix(cm,
                          target_names,
                          title='Confusion matrix',
                          cmap=None,
                          normalize=True):
    """
    given a sklearn confusion matrix (cm), make a nice plot

    Arguments
    ---------
    cm:           confusion matrix from sklearn.metrics.confusion_matrix

    target_names: given classification classes such as [0, 1, 2]
                  the class names, for example: ['high', 'medium', 'low']

    title:        the text to display at the top of the matrix

    cmap:         the gradient of the values displayed from matplotlib.pyplot.cm
                  see http://matplotlib.org/examples/color/colormaps_reference.html
                  plt.get_cmap('jet') or plt.cm.Blues

    normalize:    If False, plot the raw numbers
                  If True, plot the proportions

    """

    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy

    if cmap is None:
        cmap = plt.get_cmap('Blues')

    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]


    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")


    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
    plt.show()

# Utility funtion to load the dataset

This function allows to load the datasets for training and testing.

We are taking in this function the data already preprocessed in the other preprocessing notebook.

this function returns two variables:
* x_(train/test): this is a tensor with images
* y_(train/test): array with zeros and ones  for labels. (ex: 0 = no tumor and 1 = glioma)

to load the proper set of data we have to pass two parameters to the funtion:
* dataset:  is a string with the value "training" or "testing"  to know which set are we loaing.
* _categories: two strings in an array to know which categories are we using (ex: ["no_tumor","glioma_tumor"])

In [ ]:
def load_dataset(dataset,_categories):
    data = []
    labels = []
    path = ""
    
    if dataset != "training" or dataset == "testing":
        print(f"Error: invalid dataset type, options are 'training' or 'testing'")
        return
    
    if dataset == "training":
        print("loading preprocessed training dataset")
        path = "preprocessed/Training"
        
    if dataset == "testing":
        print("loading preprocessed testing dataset")
        path = "preprocessed/Testing"
        
    if len(_categories) != 2:
        print("Error: please select two categories, this is for a binary classifier")
        return
    
    for category in _categories:
        if category not in categories:
            print(f"Error: invalid category, options are {categories}")
            return 
        
    for category in _categories:
        label = _categories.index(category)
        cat_path=f"{path}/{category}"
        print(f"loading category {category} from path {cat_path}")
        imgs_files = glob.glob(f"{cat_path}/*")
        for img in imgs_files:
            mat = np.load(img)
            data.append(mat)
            labels.append(label)
    data = np.array(data).reshape((len(data),64,64,1))
    labels = np.array(labels)
    return data,labels

# Loading the datasets

In this section we are loading the datasets into the memory for traning and testing using our previuosly defined function.

* x_(train/test): numpy tensor with images
* y_(train/test): numpy array labels 

The dataset we are using is providing only sub set for training and testing, in order to have a validation sub set, we are splitting the test dataset into two subesets with scikit learn.

Why validation subset? this is useful when you want to monitor the model in the training process. this allows to see for example if we have over training.

In [ ]:
x_train, y_train = load_dataset("training",categories_selected)
x_test, y_test = load_dataset("training",categories_selected)

#creating an aditional validation dataset 
x_valid, x_test, y_valid, y_test = train_test_split(x_test, y_test, test_size=0.5, shuffle= True)

# Summary of datasets

Lets see some figures about our is distributed the subsets for training, validation and testing.

In [ ]:
plt.rcParams["figure.figsize"] = (20,20)
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(nrows=2, ncols=2)

ax1.set_title('Subsets images')
ax2.set_title('Training labels')
ax3.set_title('Validation labels')
ax4.set_title('Test labels')

labels = ['training', 'validation', 'test']
shapes = [x_train.shape[0],x_valid.shape[0],x_test.shape[0]]
ax1.bar(labels,shapes)

ax2.bar(categories_selected,[len(y_train)-np.count_nonzero(y_train),np.count_nonzero(y_train)])

ax3.bar(categories_selected,[len(y_valid)-np.count_nonzero(y_valid),np.count_nonzero(y_valid)])

ax4.bar(categories_selected,[len(y_test)-np.count_nonzero(y_test),np.count_nonzero(y_test)])

plt.show()

# Defining the metrics for the model.

There is a lot of metrics used in statistis to measure the performance of a model, for this exercise we took
the most importants and widely used in medicine.

1) the accuracy is the most popular for classification<a href="#11">[11]</a> , it is defined as:
$$\text{Accuracy} = \frac{TP+TN}{TP+TN+FP+FN}$$


2) Sensitivity measures the proportion of actual positives that are correctly identified as such
$$\text{Sensitivity} = \frac{TP}{TP+FN}$$

3) Specificity measures the proportion of actual negatives that are correctly identified as such 
$$\text{Sensitivity} = \frac{TN}{TN+FP}$$


4) Receiver operating characteristics (ROC), are commonly used in medical decision making <a href="#12">[12]</a> 
let's take a look for a simple explanetion <a src="https://developers.google.com/machine-learning/crash-course/classification/roc-and-auc">here</a>

Where TP = True Positives, TN = True Negatives, FP = False Positives, and FN = False Negatives.



The AUC (Area Under the Curve) for ROC and the accuracy are defined by default in tensorflow metrics, sensitivity and specificity aren't, then I defined then in funcitons that will be called in the training by tensorflow.


In [ ]:
def sensitivity_caller():
    _tp = tf.keras.metrics.TruePositives()
    _fn = tf.keras.metrics.FalseNegatives()
    @tf.function
    def sensitivity(y_true, y_pred):
        tp = _tp(y_true, y_pred)
        fn = _fn(y_true, y_pred)
        return tp/(tp+fn+K.epsilon())
    return sensitivity

def specificity_caller():
    _tn = tf.keras.metrics.TrueNegatives()
    _fp = tf.keras.metrics.FalsePositives()
    @tf.function
    def specificity(y_true, y_pred):
        tn = _tn(y_true, y_pred)
        fp = _fp(y_true, y_pred)
        return tn/(tn+fp+K.epsilon())
    return specificity

metrics = [
  tf.keras.metrics.BinaryAccuracy(name='accuracy'),
  tf.keras.metrics.AUC(name='auc'), #ROC
  sensitivity_caller(),
  specificity_caller()
]

# Machine Learning model

The next step is to define the ML model using tensorflow, for this we are using at speialized architecture for image processing based convolutional neural networks.

<img src="https://miro.medium.com/max/700/1*kkyW7BR5FZJq4_oBTx3OPQ.png"/>

Long history short, the convolutional neural networks is set of layers that allows to apply filters over the images along the multiple layer to ideantify the pattern.

<img src="https://miro.medium.com/max/700/1*ulfFYH5HbWpLTIfuebj5mQ.gif"/>


images taken from https://towardsdatascience.com/convolutional-neural-networks-explained-9cc5188c4939


In [ ]:
model = Sequential()

#hyperparameters
sdropout = 0.2
dropout = 0.2
n_filter = 2
img_size=(64,64,1)
optimizer_lr = 1e-3
dense_neurons = 8


model.add(Conv2D(2*n_filter, kernel_size=16, activation=tf.nn.relu, input_shape=img_size))
model.add(Conv2D(4*n_filter, kernel_size=8))
model.add(Activation("relu"))
model.add(AveragePooling2D(pool_size=(2,2), strides=None, padding='valid', data_format=None))

model.add(Conv2D(4*n_filter, kernel_size=8))
model.add(Activation("relu"))
model.add(AveragePooling2D(pool_size=(2, 2), strides=None, padding='valid', data_format=None))

model.add(Conv2D(8*n_filter, kernel_size=4))
model.add(Activation("relu"))
model.add(AveragePooling2D(pool_size=(2, 2), strides=None, padding='valid', data_format=None))

model.add(Flatten())
model.add(Dense(dense_neurons, activation=tf.nn.relu))
model.add(Dropout(dropout))
model.add(Dense(1, activation=tf.nn.sigmoid))

model.compile(optimizer=Adam(optimizer_lr),
              loss='binary_crossentropy',metrics = metrics)
model.summary()

In [ ]:
epochs = 20
batch_size = 10
history = model.fit(x_train, y_train, validation_data=(x_valid, y_valid), 
                    batch_size=batch_size, epochs=epochs)

In [ ]:
# list all data in history
print(history.history.keys())
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(nrows=2, ncols=2)

ax1.set_title('Loss Training/Validation')
ax2.set_title('Accuracy Training/Validation')
ax3.set_title('Sensitivity Training/Validation')
ax4.set_title('Specificity Training/Validation')

# summarize history for accuracy
ax1.plot(history.history['loss'])
ax1.plot(history.history['val_loss'])

ax2.plot(history.history['accuracy'])
ax2.plot(history.history['val_accuracy'])

ax3.plot(history.history['sensitivity'])
ax3.plot(history.history['val_sensitivity'])

ax4.plot(history.history['specificity'])
ax4.plot(history.history['val_specificity'])

ax1.set_xlabel('epoch')
ax1.set_ylabel('loss')

ax2.set_xlabel('epoch')
ax2.set_ylabel('accuracy')

ax3.set_xlabel('epoch')
ax3.set_ylabel('sensitivity')

ax4.set_xlabel('epoch')
ax4.set_ylabel('specificity')


ax1.legend(['train loss', 'valid loss'], loc='upper left')
ax2.legend(['train acc','valid acc'], loc='upper left')
ax3.legend(['train sen','valid sen'], loc='upper left')
ax4.legend(['train spe','valid spe'], loc='upper left')


plt.show()

# Analysis with the testing subset

Let's validate the models with the testing subset, for this we are going to **make a predictions** over the test subset, we are going to plot the **confusion matrix**, **calculate ROC, sensitivity** and **specificity**. 

In [ ]:
y_pred = model.predict_classes(x_test)

In [ ]:
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
plot_confusion_matrix(cm           = confusion_matrix(y_test, y_pred), 
                      normalize    = False,
                      target_names = categories_selected,
                      title        = "Confusion Matrix")

# Sensitivity and specificity
As defined on https://en.wikipedia.org/wiki/Sensitivity_and_specificity
* Sensitivity (True Positive rate) measures the proportion of positives that are correctly identified (i.e. the proportion of those who have some condition (affected) who are correctly identified as having the condition).
* Specificity (True Negative rate) measures the proportion of negatives that are correctly identified (i.e. the proportion of those who do not have the condition (unaffected) who are correctly identified as not having the condition).


**Sensitivity** = TP / (TP + FN) 

**Specificity** = TN / (TN + FP)

In [ ]:
sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)

print(f"Sensitivity = {sensitivity} ")
print(f"Specificity = {specificity} ")


## References
<a id="1">[1] </a> https://www.aans.org/en/Patients/Neurosurgical-Conditions-and-Treatments/Brain-Tumors

<a id="2">[2] </a> https://www.kaggle.com/sartajbhuvaji/brain-tumor-classification-mri

<a id="3">[3] </a>https://www.mdpi.com/2076-3417/10/6/1999

<a id="4">[4] </a>https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4879034/

<a id="5">[5] </a>https://github.com/ANTsX/ANTs

<a id="6">[6] </a>https://scikit-image.org/

<a id="7">[7] </a>https://opencv.org/

<a id="8">[8] </a>https://www.tensorflow.org/

<a id="9">[9] </a>https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html

<a id="10">[10] </a>https://www.kaggle.com/grfiv4/plot-a-confusion-matrix

<a id="11">[11] </a>https://developers.google.com/machine-learning/crash-course/classification/accuracy

<a id="12">[12] </a> https://people.inf.elte.hu/kiss/11dwhdm/roc.pdf